In [9]:
import nltk
import numpy as np
import random
import string

import bs4 as bs
import urllib.request
import re

raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/India')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

# Maintaining initial corpus for displaying answers.
# Clean the article sentence to remove extra whitespaces and reference numbers (such as "[23]")
initial_corpus = nltk.sent_tokenize(article_text)
for i in range(len(initial_corpus)):
    initial_corpus[i] = re.sub(r'\[\d+\]', '', initial_corpus[i])
    initial_corpus[i] = re.sub(r'\[\d+,\s\d+]', '', initial_corpus[i])
    initial_corpus[i] = re.sub(r'\[\w\]', '', initial_corpus[i])
    initial_corpus[i] = re.sub(r'\s+', ' ', initial_corpus[i]).strip()    
    
corpus = nltk.sent_tokenize(article_text)

for i in range(len(corpus )):
    corpus [i] = corpus [i].lower()
    corpus [i] = re.sub(r'\W',' ',corpus [i])
    corpus [i] = re.sub(r'\s+',' ',corpus [i])

wordfreq = {}
for sentence in corpus:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

import heapq
most_freq = heapq.nlargest(200, wordfreq, key=wordfreq.get)

In [10]:
word_idf_values = {}
for token in most_freq:
    doc_containing_word = 0
    for document in corpus:
        if token in nltk.word_tokenize(document):
            doc_containing_word += 1
    word_idf_values[token] = np.log(len(corpus)/(1 + doc_containing_word))

In [11]:
word_tf_values = {}
for token in most_freq:
    sent_tf_vector = []
    for document in corpus:
        doc_freq = 0
        for word in nltk.word_tokenize(document):
            if token == word:
                  doc_freq += 1
        word_tf = doc_freq/len(nltk.word_tokenize(document))
        sent_tf_vector.append(word_tf)
    word_tf_values[token] = sent_tf_vector

In [12]:
tfidf_values = []
for token in word_tf_values.keys():
    tfidf_sentences = []
    for tf_sentence in word_tf_values[token]:
        tf_idf_score = tf_sentence * word_idf_values[token]
        tfidf_sentences.append(tf_idf_score)
    tfidf_values.append(tfidf_sentences)

In [13]:
tf_idf_model = np.asarray(tfidf_values)

In [14]:
tf_idf_model = np.transpose(tf_idf_model)

# Model evaluation:

In [23]:
def clean_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'\W', ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

In [24]:
import json

with open('val_questions.json', 'r') as fp:
    val_questions = json.load(fp)
    
with open('val_answers.json', 'r') as fp:
    val_answers = json.load(fp)

In [25]:
pred_answers = dict()

for qid, question in val_questions.items():
    cleaned_question = clean_sentence(question)
    question_vector = compute_tfidf_model([cleaned_question], most_freq_tokens)

    similarity_scores = []
    sent_vec_index = 0
    for sent_vec in tuned_tfidf_model:
        similarity = 1 - sp.spatial.distance.cosine(question_vector, sent_vec)
        similarity_scores.append((sent_vec_index, similarity))
        sent_vec_index += 1
    similarity_scores.sort(key = lambda x: x[1], reverse=True)
    answer_index = similarity_scores[0][0]
    answer = initial_corpus[answer_index]

    pred_answers[qid] = initial_corpus[answer_index]

NameError: name 'compute_tfidf_model' is not defined